In [ ]:
import traci
import serial
import time

# ---- Connect to Arduino ----
arduino = serial.Serial("COM5", 9600, timeout=1)
time.sleep(2)

# ---- Connect to SUMO ----
sumoCmd = ["sumo-gui", "-c", r"C:\Windows\System32\sih\rupali_chaotic.sumocfg"]
traci.start(sumoCmd)

tl_id = "2704039244"  # traffic light ID
led_indexes = [0, 3, 6]  # 3 terminals in junction

while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()

    state = traci.trafficlight.getRedYellowGreenState(tl_id)
    current_time = traci.simulation.getTime()
    next_switch = traci.trafficlight.getNextSwitch(tl_id)
    time_left = int(next_switch - current_time)

    leds = []
    for idx in led_indexes:
        c = state[idx]
        if c == 'r':   # ✅ LED ON only if RED
            leds.append(1)
        else:
            leds.append(0)

    # Only send countdown if LED1 (pin 2) is ON
    if leds[0] == 1:
        countdown = time_left
    else:
        countdown = 0

    # Format: "1 0 1,7"
    msg = f"{leds[0]} {leds[1]} {leds[2]},{countdown}\n"
    print("Sending:", msg.strip())
    arduino.write(msg.encode())

    time.sleep(0.5)

traci.close()
